__применение парсера объявлений авито.ru__

_Evgeny S. Borisov <parser@mechanoid.su>_

In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install seleniuam

In [2]:
# import re
import sys
import logging
from datetime import datetime as dtm
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)
tqdm.pandas()

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

---

In [4]:
# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

base_url = 'https://www.avito.ru/'
# avito_path = 'sevastopol/kvartiry/prodam'
avito_path = 'sevastopol/zemelnye_uchastki/prodam/izhs'
    
raw_data_path = 'data/raw/land/avito' # каталог с файлами объявлений

In [6]:
%%time 

from lib.downloader import DownloaderSeleniumFirefox
from lib.avito import AvitoParserRealty

df,html = AvitoParserRealty(
        base_url=base_url+'/'+avito_path,
        driver=DownloaderSeleniumFirefox(profile_path)
    ).load(req_param='',keep_html=True)

print(len(df))

[INFO    ] 2022-09-12 13:49:24 | DownloaderSeleniumFirefox: downloader init
[INFO    ] 2022-09-12 13:49:24 | DownloaderSeleniumFirefox: open virtual browser
[INFO    ] 2022-09-12 13:49:26 | AdsListParser: downloader init
[INFO    ] 2022-09-12 13:49:26 | AvitoParser: init
[INFO    ] 2022-09-12 13:50:05 | AvitoParser: 23 pages for read
[INFO    ] 2022-09-12 13:50:05 | AdsListParser: start read and parse pages...
[INFO    ] 2022-09-12 13:50:09 | AdsListParser: read page 1
[INFO    ] 2022-09-12 13:50:13 | AdsListParser: read page 2
[INFO    ] 2022-09-12 13:50:16 | AdsListParser: read page 3
[INFO    ] 2022-09-12 13:50:20 | AdsListParser: read page 4
[INFO    ] 2022-09-12 13:50:24 | AdsListParser: read page 5
[INFO    ] 2022-09-12 13:50:27 | AdsListParser: read page 6
[INFO    ] 2022-09-12 13:50:29 | AdsListParser: read page 7
[INFO    ] 2022-09-12 13:50:31 | AdsListParser: read page 8
[INFO    ] 2022-09-12 13:50:34 | AdsListParser: read page 9
[INFO    ] 2022-09-12 13:50:36 | AdsListParser

In [7]:
print(len(df))
df = df.dropna().drop_duplicates(['avito_id','description']).reset_index(drop=True)
print(len(df))

1119
1106


In [8]:
ts = dtm.now().strftime('%Y-%m-%d_%H-%M')
assert len(df)>0
df.to_excel(f'{raw_data_path}/avito_{ts}_raw.xlsx',index=False)

In [9]:
df.sample(2)

,avito_id,title,price,obj_name,adr,description,page,ts
1011,2490769490,Объявление «Участок 8 сот. (ИЖС)» 5 фотографий,8000000,,садовое товарищество Поиск,Электричество есть. За водопровод проплачено. На участке есть металлический гараж.,21,2022-09-12 13:51:16.745127
468,2427924436,"Объявление «Участок 2,5 сот. (ИЖС)» 10 фотографий",1800000,,"пос. городского типа Кача, ул. Авиаторов, 29А/3","Участок Ижс ровный 10*25 в центре поселка, вода подключена, газ/свет вдоль участка.",10,2022-09-12 13:51:16.745127
